# Custom Class

In [17]:
import yfinance as yf
import pandas as pd
import matplotlib as plt
import numpy as np
from statsmodels.tsa.stattools import adfuller

import requests as rq
from bs4 import BeautifulSoup as bs

In [2]:
'''
Going to define a custom class, each instance is the information on a single stock, will then define a seperate 
analysing class to compare stocks

This class StockInfo will contain all information about a stock between a certain period specified and will 
also be able to compute relevant mathematical computations on the stock info, as well as plot some key graphs.
'''

class StockInfo():
    '''
    This class gathers the 'Adj Close' prices of the stock with ticker = ticker
    
    When you initiate the function you have the option to select a start and end date, if none are inputted,
    all possible data will be selected. The data is stored in a pandas DataFrame.
    '''
    def __init__(self, ticker: str, start = None, end = None, interval = '1d') -> pd.DataFrame:
        adj_close_prices = yf.download(ticker, start = start, end = end, interval = interval)['Adj Close'].to_frame()
        
        self.data = adj_close_prices.rename(columns = {'Adj Close' : f"{ticker}"})
        self.ticker = ticker

    def default_period(self, period):
        if not period:
            period = [self.data.index[0], self.data.index[-1], 1]
        return period

    '''
        For Displaying info about the Stock Prices
                                                    '''

    def display_data(self, period = None):
        period = self.default_period(period)
        return self.data.loc[period[0] : period[1]]

    def plot_stock_price(self, period = None):
        period, segment = self.default_period(period), self.display_data(period)
        segment.plot()

    '''
        THESE METHODS HELP YOU WORK WITH RETURNS 
                                                    '''
    
    # 'returns' function outputs the returns of the stock in a given interval
    def df_returns(self, period = None) -> pd.DataFrame:
        period = self.default_period(period)
        segment = self.data.loc[period[0] : period[1]]
        return segment.div(segment.iloc[0]).mul(period[2])

    def returns(self, period = None) -> float:
        return_df = self.df_returns(period)
        return return_df.iloc[-1] - return_df.iloc[0]

    def perc_returns(self, period = None) -> float:
        return_df = self.df_returns(period)
        return (return_df.iloc[-1] - return_df.iloc[0]) / return_df.iloc[0] * 100

    def plot_returns(self, period = None):
        return_df = self.df_returns(period)
        return_df.plot()
    


    

    '''
        THESE METHODS HELP YOU WORK WITH VOLATILITY
                                                    '''

    def variance(self, period = None):
        period = self.default_period(period)
        segment = self.data.loc[period[0] : period[1]]
        return segment.pct_change().mul(100).dropna().var()

    def sigma(self, period = None):
        return np.sqrt(self.variance(period))


    '''
        Augmented Dickey-Fuller Test
                                    '''

    def adf_test(self, period = None, graph = False):
        period = self.default_period(period)
        segment = self.data.loc[period[0] : period[1]]
        arr = segment.iloc[:,0] # Converts 1D Pandas DataFrame to series
        result = adfuller(arr)
        if graph:
            self.plot_stock_price(period)
        return f" ADF Statistic: {result[0]} \np-value: {result[1]}"
        
        
        
        
        
msft = StockInfo('MSFT')

C:\Users\DarrenJackson\Desktop\Coding\Projects\Quantitative Trading\Project_1\venviro2\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [22]:
# Get all the S&P 500 Indexes

url = 'https://stockanalysis.com/list/sp-500-stocks/'
response = rq.get(url)
soup = bs(response.text, 'html.parser')

In [39]:
bro = soup.find_all('td')
bro

[<td class="svelte-cod2gs">1</td>,
 <td class="sym svelte-cod2gs"><!-- HTML_TAG_START --><a href="/stocks/msft/">MSFT</a><!-- HTML_TAG_END --></td>,
 <td class="slw svelte-cod2gs">Microsoft Corporation</td>,
 <td class="svelte-cod2gs">2,992.91B</td>,
 <td class="svelte-cod2gs">402.79</td>,
 <td class="rr svelte-cod2gs">-0.31%</td>,
 <td class="tr svelte-cod2gs">227.58B</td>,
 <td class="svelte-cod2gs">2</td>,
 <td class="sym svelte-cod2gs"><!-- HTML_TAG_START --><a href="/stocks/aapl/">AAPL</a><!-- HTML_TAG_END --></td>,
 <td class="slw svelte-cod2gs">Apple Inc</td>,
 <td class="svelte-cod2gs">2,803.63B</td>,
 <td class="svelte-cod2gs">181.56</td>,
 <td class="rr svelte-cod2gs">-0.41%</td>,
 <td class="tr svelte-cod2gs">385.71B</td>,
 <td class="svelte-cod2gs">3</td>,
 <td class="sym svelte-cod2gs"><!-- HTML_TAG_START --><a href="/stocks/goog/">GOOG</a><!-- HTML_TAG_END --></td>,
 <td class="slw svelte-cod2gs">Alphabet Inc.</td>,
 <td class="svelte-cod2gs">1,754.12B</td>,
 <td class="s

In [42]:
tickers = []
for td in bro:
    a_tags = td.find('a')
    if a_tags:
        tickers.append(a_tags.text)

In [43]:
tickers

['MSFT',
 'AAPL',
 'GOOG',
 'AMZN',
 'NVDA',
 'META',
 'BRK.B',
 'GOOGL',
 'LLY',
 'TSLA',
 'AVGO',
 'JPM',
 'UNH',
 'WMT',
 'V',
 'MA',
 'XOM',
 'JNJ',
 'PG',
 'HD',
 'MRK',
 'COST',
 'ABBV',
 'ORCL',
 'CVX',
 'CRM',
 'BAC',
 'AMD',
 'KO',
 'NFLX',
 'ADBE',
 'PEP',
 'ACN',
 'MCD',
 'TMO',
 'LIN',
 'ABT',
 'DIS',
 'CSCO',
 'TMUS',
 'INTC',
 'WFC',
 'DHR',
 'INTU',
 'VZ',
 'QCOM',
 'IBM',
 'CMCSA',
 'GE',
 'CAT',
 'UBER',
 'AMAT',
 'PFE',
 'NOW',
 'AXP',
 'AMGN',
 'UNP',
 'TXN',
 'MS',
 'PM',
 'SPGI',
 'ISRG',
 'SYK',
 'HON',
 'LOW',
 'BKNG',
 'COP',
 'NKE',
 'GS',
 'BA',
 'PLD',
 'T',
 'RTX',
 'ELV',
 'BLK',
 'LRCX',
 'NEE',
 'PANW',
 'MDT',
 'SCHW',
 'TJX',
 'PGR',
 'ETN',
 'VRTX',
 'UPS',
 'C',
 'SBUX',
 'ADP',
 'LMT',
 'REGN',
 'CB',
 'BMY',
 'DE',
 'CI',
 'MMC',
 'MDLZ',
 'BSX',
 'CVS',
 'ADI',
 'GILD',
 'BX',
 'MU',
 'FI',
 'KLAC',
 'AMT',
 'ZTS',
 'SNPS',
 'HCA',
 'ANET',
 'WM',
 'EQIX',
 'SHW',
 'CDNS',
 'ICE',
 'ITW',
 'CME',
 'GD',
 'SO',
 'CSX',
 'MO',
 'DUK',
 'CMG',
 'MAR',

In [44]:
len(tickers)

503